# Solving the SAT problem using QAOA

In order to embed an instance of the SAT problem into a Hamiltonian $H_c$, we use the following mapping:

$$
    x_j \rightarrow P^j_0 \\ 
    \overline{x}_j \rightarrow P^j_1 \\ 
    \lor \rightarrow \otimes \\
    \land \rightarrow +
$$

In QAOA, we prepare the state
$$
    |\psi(\boldsymbol{\alpha}, \boldsymbol{\beta})\rangle = e^{i \beta_k H_c} e^{i \alpha_k H_m} \ldots e^{i \beta_1 H_c} e^{i \alpha_1 H_m} |\boldsymbol{0}\rangle,
$$

where *the mixing operator* is chosen to be $H_m = -\sum_j{\sigma_x^j},\, \boldsymbol{\alpha} = \{\alpha_i\},\, \boldsymbol{\beta} = \{\beta_i\}$ and $\alpha_i, \beta_i \in [0, 2\pi)$.

We minimize the expectation value of $H_c$ over the sets of parameters $\boldsymbol{\alpha}, \boldsymbol{\beta}$, e.g. we calculate:

$$
    E_{min} = \min_{\boldsymbol{\alpha}, \boldsymbol{\beta}}{\langle \psi(\boldsymbol{\alpha}, \boldsymbol{\beta})| H_c |\psi(\boldsymbol{\alpha}, \boldsymbol{\beta})\rangle}
$$

We also search for the sets of parameters
$$
    \widetilde{\boldsymbol{\alpha}}, \widetilde{\boldsymbol{\beta}} = \arg\min_{\boldsymbol{\alpha}, \boldsymbol{\beta}}{\langle \psi(\boldsymbol{\alpha}, \boldsymbol{\beta})| H_c |\psi(\boldsymbol{\alpha}, \boldsymbol{\beta})\rangle}
$$
such that 
$$
    \langle \psi(\widetilde{\boldsymbol{\alpha}}, \widetilde{\boldsymbol{\beta}})| H_c |\psi(\widetilde{\boldsymbol{\alpha}}, \widetilde{\boldsymbol{\beta}})\rangle = E_{min}.
$$

In this tutorial, we use the *ProjectQ* package for QAOA calculations.

After we found the smallest eigenvalue, we compare the corresponding eigenvector with the exact solution for a SAT instance.

To install ProjectQ, run

```
pip3 install projectq
```

or, if it fails,

```
pip3 install projectq --global-option=--without-cppsimulator
```

In [1]:
import projectq
from projectq.ops import All, Measure, QubitOperator, TimeEvolution
from projectq.ops import X, Y, Z, H, Rx, Ry, Rz, CNOT, Swap

In [2]:
import numpy as np
from numpy import kron
import scipy
from scipy import linalg as la
from scipy.optimize import minimize
import random
from functools import reduce
import time

In [3]:
P_0 = np.array([[1,0],[0,0]]) # |0><0|
P_1 = np.array([[0,0],[0,1]]) # |1><1|
X_np = np.array([[0,1],[1,0]]) # X Pauli matrix
Z_np = np.array([[1,0],[0,-1]]) # Z Pauli matrix
I_np = np.array([[1,0],[0,1]]) # 2x2 identity matrix 

In [4]:
def cnf_unique_generator(sat, variables_number, clauses_number):
    '''
    Generates a list of unique CNFs
    ''' 
    cnf_list = []
    
    while len(cnf_list) <= clauses_number-1:
        a = np.array(sorted(random.sample(range(1, variables_number+1), sat)))
        b = np.array([(-1)**random.randint(0, 1) for i in range(sat)])
        clause = list(a*b)
        
        if clause not in cnf_list:
            cnf_list.append(clause)
    
    return cnf_list

In [5]:
print("An example of a 3SAT instance: \n\n", cnf_unique_generator(sat=3, variables_number=4, clauses_number=5))

An example of a 3SAT instance: 

 [[-1, 3, 4], [1, 2, 3], [-1, 2, 4], [-2, -3, 4], [-1, -3, -4]]


In [28]:
def cnf_to_hamiltonian(cnf, sat):
    '''
    Transforms a CNF to a Hamiltonian for ProjectQ
    ''' 
    def projector(qubit, index):
        if index == 1:
            return 0.5 * (QubitOperator("") + QubitOperator("Z"+str(qubit)))
        if index == -1:   
            return 0.5 * (QubitOperator("") - QubitOperator("Z"+str(qubit)))
    
    #sat = len(cnf[0])

    hamiltonian = projector(abs(cnf[0][0])-1, np.sign(cnf[0][0]))
    for i in range(1, sat):
        hamiltonian *= projector(abs(cnf[0][i])-1, np.sign(cnf[0][i]))

    for clause in cnf[1:]:
        
        ham = projector(abs(clause[0])-1, np.sign(clause[0]))
        for i in range(1, sat):
            ham *= projector(abs(clause[i])-1, np.sign(clause[i]))

        hamiltonian += ham

    return hamiltonian   

In [16]:
cnf = cnf_unique_generator(sat=3, variables_number=3, clauses_number=3)
print("An example of a 3SAT Hamiltonian:\n\n", cnf_to_hamiltonian(cnf=cnf, sat=3))

An example of a 3SAT Hamiltonian:

 0.375 I +
-0.125 Z1 Z2 +
-0.125 Z0 +
0.375 Z0 Z1 Z2 +
0.125 Z2 +
0.125 Z1 +
0.125 Z0 Z2 +
0.125 Z0 Z1


In [17]:
def cnf_to_matrix(sat_instance, num_bits, pauli=Z_np):
    '''
    Creates a SAT-Hamiltonian as a NumPy array. pauli specifies which basis to use
    '''    
    H = np.zeros((2**num_bits, 2**num_bits))
    for clause in sat_instance:
        term_list = [I_np] * num_bits
        for i in clause:
            term_list[abs(i) - 1] = 0.5 * (I_np + pauli * np.sign(i))
        H += reduce(kron, term_list)
    return H

In [18]:
def mixing_operator(variables_number):
    '''
    Creates the mixing operator H_m for ProjectQ
    ''' 
    operator = QubitOperator('X0')
    for k in range(1, variables_number):
        operator += QubitOperator('X' + str(k))
        
    return -operator

In [19]:
print("An example of a mixing operator: \n\n", mixing_operator(variables_number=3))

An example of a mixing operator: 

 -1.0 X0 +
-1.0 X1 +
-1.0 X2


In [20]:
def mixing_operator_matrix(variables_number):
    '''
    Creates the mixing operator H_m as a NumPy array 
    ''' 
    sum_operator = np.zeros((2**variables_number,2**variables_number))
    
    for i in range(variables_number):
        
        operator_ini = -X_np
        
        for j in range(i):
            operator_ini = np.kron(I_np, operator_ini)
        
        for k in range(i, variables_number-1):
            operator_ini = np.kron(operator_ini, I_np)
        
        sum_operator += operator_ini
        
    return sum_operator

In [21]:
def optimize_sat_instance(sat, hamiltonian, mixer, variables_number, clauses_number, sequence_length, method, engine):
    '''
    Minimizes the expectation value of a given Hamiltonian
    ''' 
    def function(x):
        
        # allocate a quantum register in the |00...0> state
        quantum_state = engine.allocate_qureg(variables_number)
        
        # transform to the |++...+> state
        for i in range(variables_number):
            H | quantum_state[i]
        
        # parameters \alpha and \beta for QAOA
        parameters = x
        
        # apply the sequence of e^{-i*\beta*H_m} e^{-i*\alpha*H_c} operators
        for i in range(sequence_length): 
            # apply unitary evolution e^{-i*parameter*operator}
            TimeEvolution(parameters[2*i], hamiltonian) | quantum_state
            TimeEvolution(parameters[2*i+1], mixer) | quantum_state
        
        # send all the gates to the backend
        engine.flush()
        
        # calculate the expectation value
        expectated_value = engine.backend.get_expectation_value(hamiltonian, quantum_state)
        
        # deallocate the quantum register
        All(Measure) | quantum_state

        return expectated_value
    
    initial_parameters = [0.25*np.pi for i in range(2*sequence_length)]
    
    bounds = [(0, 2*np.pi) for i in range(2*sequence_length)]
    
    result = minimize(function, initial_parameters, method=method, bounds=bounds)
    
    return result

This function takes the sets $\boldsymbol{\alpha}, \boldsymbol{\beta}$ and constructs $|\psi(\boldsymbol{\alpha}, \boldsymbol{\beta})\rangle = e^{i \beta_k H_c} e^{i \alpha_k H_m} \ldots e^{i \beta_1 H_c} e^{i \alpha_1 H_m} |\boldsymbol{0}\rangle$ as a NumPy array:

In [22]:
def construct_vector(variables_number, mixing_operator_matrix, hamiltonian_matrix, sequence_length, parameters):
    '''
    Creates the approximate ground eigensate as a NumPy vector
    ''' 
    # |+> state
    vector = np.array([1]*(2**variables_number)) / np.sqrt(2**variables_number)
    
    # apply the sequence of operators
    for i in range(sequence_length):
        vector = la.expm(-1j*parameters[2*i+1]*mixing_operator_matrix).dot(la.expm(-1j*parameters[2*i]*hamiltonian_matrix)).dot(vector)
    
    return vector

Suppose that the SAT instance is satisfiable, then the solution vector lies in the span of all ground eigenvectors.

The *check_overlap(...)* function finds the set of ground eigenvectors of a given Hamiltonian, $\{|\varphi_1\rangle, \ldots, |\varphi_l\rangle \}$, and then calculates the overlap between the approximate eigenvector $|\psi(\widetilde{\boldsymbol{\alpha}}, \boldsymbol{\widetilde{\beta}})\rangle$ and this set as

$$
    overlap = \sum_j{|\langle\varphi_j|\psi(\widetilde{\boldsymbol{\alpha}}, \boldsymbol{\widetilde{\beta}})\rangle|^2}
$$

This quantity shows how far our solution is from the exact one.

In [23]:
def check_overlap(target_vector, hamiltonian_matrix, variables_number):
    '''
    Calculates the overlap between the approximate eigenstate and the span of exact eigenstates
    ''' 
    eigenvalues, eigenvectors = la.eig(hamiltonian_matrix)
    ground_eigenvectors_indices = np.where(eigenvalues == min(eigenvalues))
        
    overlap = 0
    for l in ground_eigenvectors_indices[0]:
        overlap += abs(target_vector.conjugate().transpose().dot(eigenvectors[l]/np.linalg.norm(eigenvectors[l])))**2 
    
    return overlap

## Running an Example 

If the approximate eigenvalue is smaller than $1$, then we consider that the SAT instance is satisfiable.

In [26]:
variables_number = 4
clauses_number = 8
sequence_length = 5
sat = 3

# optimization method
method = 'L-BFGS-B'

# set the default backend for ProjectQ
engine = projectq.MainEngine()

# generate a SAT instance
cnf = cnf_unique_generator(sat, variables_number, clauses_number)

# generate a Hamiltonian and a mixing operator for ProjectQ
hamiltonian = cnf_to_hamiltonian(cnf, sat)
mixer = mixing_operator(variables_number)

# generate a Hamiltonian and a mixing operator as the NumPy arrays
hamiltonian_matrix = cnf_to_matrix(cnf, variables_number)
mixer_matrix = mixing_operator_matrix(variables_number)

In [27]:
# optimize the Hamiltonian, obtain the ground eigenvalue and the sets of parameters \alpha and \beta
start_time = time.time()
result = optimize_sat_instance(sat,  hamiltonian, mixer, variables_number, clauses_number, sequence_length, method, engine)
finish_time = time.time() - start_time

# construct the ground eigenvector using the \alpha and \beta parameters
parameters = result.x
vec_opt = construct_vector(variables_number, mixer_matrix, hamiltonian_matrix, sequence_length, parameters)

print("Time:", finish_time)
print("Approximate eigenvalue:", result.fun)
print("Exact eigenvalue:", la.eigh(hamiltonian_matrix)[0][0])
print("Overlap between approxiate and exact eigenvector:", check_overlap(vec_opt, hamiltonian_matrix, variables_number))
#print("Parameters:", parameters)

Time: 163.4611372947693
Approximate eigenvalue: 0.008393808893097142
Exact eigenvalue: 0.0
Overlap between approxiate and exact eigenvector: 0.9920214114585775
